In [24]:
import json
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
from results import Results

In [25]:
targetSizeWidth = 112
targetSizeHeight = 112
targetSize = (targetSizeWidth, targetSizeHeight)
batchSize = 32
directory_filename='C:/Users/juano/Documents/Master MBD/Segundo_Cuatrimestre/Datos no estructurados/RepoGit/FaceDetection/TRModel'
testDirectory='./test'

In [26]:
def load(filename: str):
    """Loads a trained CNN model and the corresponding preprocessing information.

    Args:
        filename: Relative path to the file without the extension.

    """
    # Load Keras model
    model = tf.keras.models.load_model(filename + '.h5')

    # Load base model information
    with open(filename + '.json') as f:
        model_name = json.load(f)
    return model

cnn=load(directory_filename)

In [27]:
def predict(model,test_dir,dataset_name:str, save: bool = True):
    """Evaluates a new set of images using the trained CNN.

    Args:
        test_dir: Relative path to the validation directory (e.g., 'dataset/test').
        dataset_name: Dataset descriptive name.
        save: Save results to an Excel file.

    """
    # Configure loading and pre-processing functions
    print('Reading test data...')
    test_datagen = ImageDataGenerator(rescale=1./255)

    test_generator = test_datagen.flow_from_directory(
        './test',
        target_size = targetSize,
        batch_size=1,  # A batch size of 1 ensures that all test images are processed
        class_mode='categorical',
        shuffle=False
    )

    # Predict categories
    predictions =model.predict(test_generator)
    predicted_labels = np.argmax(predictions, axis=1).ravel().tolist()
    print(type(test_generator.class_indices))
   
    # Format results and compute classification statistics
    results = Results(test_generator.class_indices, dataset_name=dataset_name)
    accuracy, confusion_matrix, classification = results.compute(test_generator.filenames, test_generator.classes,
                                                                     predicted_labels)
    # Display and save results
    results.print(accuracy, confusion_matrix)

    # if save:
    #     results.save(confusion_matrix, classification, predictions)

In [28]:
predict(cnn,testDirectory,'Validation')

Reading test data...
Found 580 images belonging to 3 classes.
580/580 [==============================] - 3s 6ms/step
<class 'dict'>


CLASSIFICATION RESULTS (VALIDATION)

Confusion matrix

KNOWN/PREDICTED  JuanM  JuanO  Pablo
JuanM            462.0    0.0    0.0
JuanO             71.0    0.0    0.0
Pablo             47.0    0.0    0.0

Accuracy:  0.7965517241379311
